In [1]:
import PyPDF2, mysql.connector, re
from mysql.connector import Error

Obtendo as páginas do pdf em formato txt

In [2]:
with open('dados.pdf', 'rb') as pdf:
    leitor = PyPDF2.PdfFileReader(pdf)
    pagina1 = leitor.getPage(0)
    pagina2 = leitor.getPage(1)
    print(pagina1.extractText())
    print(pagina2.extractText())

John Doe
Lorem ipsum dolor sit amet, consectetur adipiscing elit,
sed do eiusmod tempor incididunt ut labore et dolore
magna aliqua. Ut enim ad minim veniam, quis nostrud
exercitation ullamco laboris nisi ut aliquip ex ea
commodo consequat. Duis aute irure dolor in
reprehenderit in voluptate velit esse cillum dolore eu
fugiat nulla pariatur. Excepteur sint occaecat cupidatat
non proident, sunt in culpa qui officia deserunt mollit
anim id est laborum.
Lorem ipsum dolor sit amet, consectetur adipiscing elit,
sed do eiusmod tempor incididunt ut labore et dolore
magna aliqua. Ut enim ad minim veniam, quis nostrud
exercitation ullamco laboris nisi ut aliquip ex ea
commodo consequat. Duis aute irure dolor in .com
prehenderit in voluptate velit esse cillum dolore eu
fugiat nulla pariatur. Excepteur sint occaecat cupidatat
non proident, sunt in culpa qui officia deserunt mollit
anim id est laborum.
Lorem ipsum dolor sit amet, consectetur adipiscing elit,
sed do eiusmod tempor incididunt ut lab

Encontrando os emails

In [3]:
# Lista de possíveis domínios de emails presentes no texto (incluindo o domínio fictício mail)
lista_de_emails = ['hotmail', 'gmail', 'outlook', 'yahoo', 'mail']

pag1 = pagina1.extract_text().split()
pag2 = pagina2.extract_text().split()
pags = pag1 + pag2

emails_encontrados = []
for palavra in pags:
    for dominio in lista_de_emails:
        if palavra.find(dominio) != -1:
            emails_encontrados.append(palavra)

n_emails = len(emails_encontrados)
print(emails_encontrados)

['john.doe@mail.com', 'john.doe@mail.com']


Encontrando os números de telefone

In [4]:
numeros_encontrados = re.findall("\(?\d+\)?[ ]?\d+[-. ]?\d+", pagina1.extract_text() + pagina2.extract_text())
n_numeros = len(numeros_encontrados)
print(numeros_encontrados)

['(11) 11111-1111', '(11) 11111-1111']


Verificando se a quantidade de emails e telefones são iguais

In [5]:
print(n_numeros == n_emails)

True


Criando a tabela Contatos para inserir os dados

In [6]:
try:
    """ Utilizei user e password diferentes,
        pois instalei o mysql no meu pc para verificar
        se os códigos desenvolvidos estavam funcionando corretamente.
    """

    conexao = mysql.connector.connect(host = 'localhost',
                                         database = 'mydatabase',
                                         user = 'root',
                                         password = '1234')
    Create_Table =  """CREATE TABLE Contatos( 
                    Id INT AUTO_INCREMENT NOT NULL,
                    Email VARCHAR(100) NOT NULL,
                    Telefone VARCHAR(15) NOT NULL,
                    PRIMARY KEY (Id))"""

    cursor = conexao.cursor()
    result = cursor.execute(Create_Table)
    print("Tabela criada com sucesso.")

except Error as e:
    print(e)
finally:
    if conexao.is_connected():
        cursor.close()
        conexao.close()
        print("Conexão com MySQL finalizada")

Tabela criada com sucesso.
Conexão com MySQL finalizada


Inserindo os dados na tabela

In [7]:
try:
    """ Utilizei user e password diferentes,
        pois instalei o mysql no meu pc para verificar
        se os códigos desenvolvidos estavam funcionando corretamente.
    """

    conexao = mysql.connector.connect(host = 'localhost',
                                         database = 'mydatabase',
                                         user = 'root',
                                         password = '1234')

    cursor = conexao.cursor()
    for i in range(n_emails):
        insert = "INSERT INTO Contatos (Email, Telefone) VALUES (%s, %s)"
        val = (emails_encontrados[i], numeros_encontrados[i])
        cursor.execute(insert, val)
    conexao.commit()
    print("Inserção feita com sucesso.")

except Error as e:
    print(e)
finally:
    if conexao.is_connected():
        cursor.close()
        conexao.close()
        print("Conexão com MySQL finalizada")

Inserção feita com sucesso.
Conexão com MySQL finalizada


Verificando a inserção

In [8]:
try:
    """ Utilizei user e password diferentes,
        pois instalei o mysql no meu pc para verificar
        se os códigos desenvolvidos estavam funcionando corretamente.
    """

    conexao = mysql.connector.connect(host = 'localhost',
                                         database = 'mydatabase',
                                         user = 'root',
                                         password = '1234')

    cursor = conexao.cursor()
    cursor.execute("SELECT * FROM Contatos")
    resultado = cursor.fetchall()

    for linha in resultado:
        print(linha)
except Error as e:
    print(e)
finally:
    if conexao.is_connected():
        cursor.close()
        conexao.close()
        print("Conexão com MySQL finalizada")

(1, 'john.doe@mail.com', '(11) 11111-1111')
(2, 'john.doe@mail.com', '(11) 11111-1111')
Conexão com MySQL finalizada
